In [1]:
"""
Author: Christian Bouwense

Program that gets the revision for a page by user for a certain time interval and measures burstiness.
"""

import time
import random
import datetime as dt
import mwapi
import operator
import numpy as np
import dateutil.parser as dup
from matplotlib import pyplot as plt

generate_sample() creates a sample of data based on the constraints given to it; this way I was able to test out my code which implemented Barabasi's formulas to measure burstiness. You can create different sizes of interevent times, and guage how often they should switch (change the effects of memory of the system). You do this by setting the length and probability of short interevent times occuring, and the same for long interevent times.

The function takes seven arguments, but they are pretty simple to understand.

n: Sample size
small_lower: Lower bound of short interevent times.
small_upper: Upper bound of short interevent times.
large_lower: Lower bound of long interevent times.
large_upper: Upper bound of long interevent times.
stop_short: Probability that, after a short interevent time, there will be a long interevent time. [0, 1]
stop_long: Probability that, after a long interevent time, there will be a short interevent time. [0, 1]
print_results: Boolean which defaults to false.

In [2]:
def generate_sample(continue_short, continue_long, n=10000, small_lower=0, small_upper=0.1, large_lower=500, large_upper=600):
    just_did_short = True
    # Initiate with a small interevent time 
    sample = [small_lower + np.mean([small_lower, small_upper])]

    for i in range(1, n):
        # The random number generate to check if we change type of interevent time (long to short or short to long)
        change_chance = random.random()
        # If we just did a short interevent time and we rolled a number in the probability to stay, make another small.
        # Or, if we just did a long time but we did not roll another long, make a small.
        if (just_did_short and change_chance <= continue_short) or (not just_did_short and change_chance > continue_long):
            just_did_short = True
            sample.append(random.uniform(small_lower, small_upper))
        else:
            just_did_short = False
            sample.append(random.uniform(large_lower, large_upper))
            
    return sample

In [3]:
# Generates sample of data that is bursty due to memory
bursty_sample = generate_sample(0.99, 0.99)
# Generates sample of data that is not bursty
non_bursty_sample = generate_sample(0.5, 0.5)

These are the arguments we are going to pass to the MediaWiki API. It specifies that we want to query the article titled "Game of Thrones" for revisions. We ask for the timestamp and user of each revision. The maximum amount of revisions any lay user is allowed per request is 500, so we request the "max" amount (in order to get all the revisions we just make a bunch of queries until we have all the revisions). We also specify that we only want the revisions made between two certain dates.

In [4]:
# Information specifying article and time interval to look at
article_title = 'Game of Thrones'

today = dt.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%dT%H:%M:%SZ')

action = 'query'
prop = 'revisions'
rv_prop = 'timestamp|user'
rv_limit = 'max'
rv_start = today
rv_end = '2010-08-05T00:00:00Z'

mwapi.Session connects to the English Wikipedia database, and we send a GET REST request using the mwapi.

In [5]:
# Connect to Wikipedia
session = mwapi.Session('https://en.wikipedia.org', user_agent='cbouwense')

# Query Wikipedia for revisions on the supplied article
# The result is stored into the dictionary "rev_dict"
rev_dict = session.get(action=action,
                       prop=prop,
                       rvprop=rv_prop,
                       titles=article_title,
                       rvlimit=rv_limit,
                       rvstart=today,
                       rvend=rv_end)

We will store the revisions made by each user in a dictionary: the keys will be each user and the values will be the amount of revisions each user made.

In [6]:
revisions_by_user = {}

This is a little weird, but necessary. We need the pageID of the article we are analyzing to use when stripping the revisions out of the JSON it gives us. If that didn't make sense it will later.

In [7]:
# Find page_id for selected article
for keys in rev_dict['query']['pages'].keys():
    page_id = keys

In [8]:
# Go through the timestamps for each revision made.
# If the timestamp is already a key in our dictionary, increment that key value by 1.
# Else, create a new key for that year in our dictionary and set it to 1
rev_timestamps = []
for props in rev_dict['query']['pages'][str(page_id)]['revisions']:
    if (props['user'] not in revisions_by_user):
        revisions_by_user[props['user']] = 1
    else:
        revisions_by_user[props['user']] += 1

    timestamp = dup.parse(props['timestamp'])
    rev_timestamps.append(timestamp)

As mentioned before, the maximum revisions a user can get from a single query is 500. If there are more than 500 revisions for the query, then there will be a key in the JSON named "continue" and "rvcontinue." We can use the value of "rvcontinue" to pick up where we left off and get revisions 501-1000 (assuming there are that many). We do this until all the revisions are received.

In [9]:
# Check if there is a section named "continue".
# If there is, that means the query did not get all the data
# because of the per-user query limits.
print ("Retrieving data from Wikipedia.")
print ("This could take up to a few minutes depending on the article...")
while 'continue' in rev_dict:
    continue_val = rev_dict['continue']['rvcontinue']
    rev_dict = session.get(action=action,
                           prop=prop,
                           rvprop=rv_prop,
                           titles=article_title,
                           rvlimit=rv_limit,
                           rvstart=today,
                           rvend=rv_end,
                           rvcontinue=continue_val)
    for props in rev_dict['query']['pages'][str(page_id)]['revisions']:
        if 'user' in props:
            if (props['user'] not in revisions_by_user):
                revisions_by_user[props['user']] = 1
            else:
                revisions_by_user[props['user']] += 1
        timestamp = dup.parse(props['timestamp'])
        rev_timestamps.append(timestamp)
        
# List of tuples of revisions made by user for page
sorted_user_revisions = sorted(revisions_by_user.items(), key=operator.itemgetter(1))[::-1]

Retrieving data from Wikipedia.
This could take up to a few minutes depending on the article...


One of the ways of measuring burstiness is analyzing the distribution of the time between events in the system (interevent times). In order to do this, we must enumerate all the interevent times.

In [10]:
# Enumerate the times between events into a list
# Also calculate the summation for M while you're at it
interevent_times = []
summation = 0
for i in range(0, len(rev_timestamps)-1):
    interevent_times.append((rev_timestamps[i] - rev_timestamps[i+1]).total_seconds())

Now that we have the interevent times, we need to find their mean and standard deviation. Luckily for us, the Python module Numpy has functions for that built in!

In [11]:
# Calculate interevent time mean and standard deviation
interevent_mean = (np.mean(interevent_times))
interevent_std_dev = (np.std(interevent_times))

The final step is to use the mean and standard deviation. The measure of burstiness by interevent times is defined as follows.
$$B \ \triangleq\ \frac{\sigma_{\tau}-m_{\tau}}{\sigma_{\tau}+m_{\tau}}$$

In [12]:
B = ((interevent_std_dev - interevent_mean) / (interevent_std_dev + interevent_mean))
print ("B = %s" % B)

B = 0.433187253169


The value of B ranges from -1 to 1: the closer to -1 the more regular the events occur, closer to 0 means the closer they are to being completely random, and the closer to 1 the more bursty.

There is a different angle that we can measure burstiness with: memory. At first it may sound the same but it is actually completely independent of interevent time distribution. Memory means that short interevent times are really likely to happen after short ones, and long ones occur after long ones.  

In order to measure this, we will need the mean and standard deviation of interevent times [1, n-1], and [2, n] (assuming a total of n interevent times).

In [13]:
mean_1 = np.mean(interevent_times[0:len(interevent_times)-1])
mean_2 = np.mean(interevent_times[1:len(interevent_times)])
std_dev_1 = np.std(interevent_times[0:len(interevent_times)-1])
std_dev_2 = np.std(interevent_times[1:len(interevent_times)])

The coefficient of burstiness due to memory is defined as

$$M \ \triangleq\ \frac{1}{n_{\tau} - 1}\sum_{i=1}^{n_{\tau}-1} \frac{(\tau_{i}-m_{1})(\tau_{i+1}-m_{2})}{\sigma_{1}\sigma_{2}}$$

In [14]:
for i in range(0, len(interevent_times)-1):
    tau_i = interevent_times[i]
    tau_i_plus_one = interevent_times[i+1]
    summation_term = (((tau_i - mean_1) * (tau_i_plus_one - mean_2)) / (std_dev_1 * std_dev_2)) 
    summation += summation_term

M = (1 / (len(interevent_times) - 1)) * summation 

print ("M = %s" % M)

M = 0.100900969388
